<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/eceg_koblitz_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter

In [2]:
# Parameters
p = 751  # prime modulus
a = -1    # coefficient of x in the elliptic curve
b = 188    # constant in the elliptic curve

# ECEG Class

In [111]:
import random
import sympy as sp
import json
import base64

class EllipticCurveElGamal:
  def __init__(self, a, b, p, k, B=None):
    self.p = p # primer number
    self.a = a # alpha
    self.b = b # beta

    self.k = k
    self.PointB = B

    self.pointP = None
    self.pointQ = None


    self.char_to_num_dict = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17,
    'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25,
    'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33,
    'y': 34, 'z': 35, '.': 36, '#': 37, '*': 38, '/': 39, '-': 40
    }

    self.num_to_char_dict = {v: k for k, v in self.char_to_num_dict.items()}

    self.k_koblitz = max(self.char_to_num_dict.values()) + 1  # max 24 for 35 char

  def example_function(self):
      print(f"Using persistent k value: {self.k}")

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  def is_on_curve(self, x, y):
    return self.elliptic_curve_equation(x)  == (y**2) % p

  # def generate_random_number(self):
  #   self.k = random.randint(1, self.p - 1)
  #   return self.k

  def generate_random_point(self):
    while True:
      x = random.randint(1, self.p - 1)
      y = random.randint(1, self.p - 1)
      if self.is_on_curve(x, y):
        return (x, y)

  def mod_inverse(self, a, p):
      if a == 0:
          raise ValueError("Inverse does not exist for 0.")
      return pow(a, p - 2, p)

  def calculate_slope_mod(self, P, Q):
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 == y2:
        # Use the formula for point doubling when P == Q
        numerator = (3 * x1**2 + self.a) % self.p
        denominator = (2 * y1) % self.p
    else:
        # Use the formula for regular slope when P != Q
        numerator = (y2 - y1) % self.p
        denominator = (x2 - x1) % self.p

    if denominator == 0:
        raise ValueError("Slope is undefined (denominator is zero).")

    # Compute the slope as (numerator / denominator) % p, which is
    # numerator * mod_inverse(denominator, p) % p
    slope = (numerator * self.mod_inverse(denominator, self.p)) % self.p
    return slope

  def calculate_add_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - x1 - x2) % self.p
    return xr

  def calculate_add_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    x2, y2 = Q
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_addition(self, P, Q):
    m = self.calculate_slope_mod(P, Q)

    xr = self.calculate_add_xr_mod(P, Q, m)

    yr = self.calculate_add_yr_mod(P, Q, m, xr)

    R = (xr, yr)

    return R

  def calculate_dob_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - (2 * x1)) % self.p
    return xr

  def calculate_dob_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_doubling(self, P):

    m = self.calculate_slope_mod(P, P)

    xr = self.calculate_dob_xr_mod(P, P, m)

    yr = self.calculate_dob_yr_mod(P, P, m, xr)

    R = (xr, yr)

    return R

  def calculate_point_multiplication(self, P, k):
    if k == 0:
      return None
    elif k == 1:
      return P

    # Initialize R to be the point at infinity, often represented as None in this context
    R = None
    Q = P  # Start with Q as P

    # Double-and-add method
    while k > 0:
        if k % 2 == 1:  # If k is odd, add Q to the result
            if R is None:
                R = Q  # R is the point at infinity initially
            else:
                R = self.calculate_point_addition(R, Q)
        Q = self.calculate_point_addition(Q, Q)  # Double the point Q
        k //= 2  # Move to the next bit

    return R

  def calculate_point_subtract(self, P, Q):
    # Find the inverse of point Q (x_Q, y_Q) -> (x_Q, -y_Q mod p)
    Q_inv = (Q[0], (-Q[1]) % self.p)

    # Subtract P - Q by adding P and Q_inv
    R = self.calculate_point_addition(P, Q_inv)

    return R

  def generate_public_key(self, B, private_key):
    public_key = self.calculate_point_multiplication(B, private_key)
    return public_key

  def enryption(self, M, public_key):
    # k = random.randint(1, self.p - 1) # 1 < k < p - 1

    C1 = self.calculate_point_multiplication(self.PointB, self.k)
    C2 = self.calculate_point_addition(M, self.calculate_point_multiplication(public_key, self.k))

    return (C1, C2)

  def decryption(self, C, private_key):
    C1, C2 = C

    _p = self.calculate_point_multiplication(C1, private_key)

    plain = self.calculate_point_subtract(C2, _p)

    return plain

  # Function to map char to num
  def char_to_num(self, char):
      # Ensure the input is a string; if not, convert it
      char = str(char)

      # Check if the string exists in the dictionary
      if char in self.char_to_num_dict:
          return self.char_to_num_dict[char]
      else:
          # Handle the expanded character set (0-40)
          raise ValueError(f"Character '{char}' is not valid. Please use '0-9', 'a-z', or other valid characters.")


  # Function to map num to char
  def num_to_char(self, num):
      # Ensure that the input is an integer; if it's a string digit, convert to integer
      if isinstance(num, str) and num.isdigit():
          num = int(num)

      # Now we expect num to be an integer
      if isinstance(num, int):
          if num in self.num_to_char_dict:  # Handle numbers 0-40 by converting them using the dictionary
              return self.num_to_char_dict[num]
          else:
              raise ValueError(f"Number '{num}' is out of the valid range (0-40).")
      else:
          raise ValueError(f"Input '{num}' is not valid. Please provide a valid number.")


  def message_to_koblitz(self, message):
      # Convert the message to a list of characters
      chars = list(message)

      koblitz = []
      for char in chars:
          num = self.char_to_num(char)
          koblitz.append(num)

      return koblitz

  def koblitz_to_message(self, koblitz):
      message = []
      for num in koblitz:
          char = self.num_to_char(num)
          message.append(char)

      return ''.join(message)

  def koblitz_encode(self, m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * self.k_koblitz + num
        rhs = (x**3 + self.a * x + self.b) % self.p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if sp.is_quad_residue(rhs, self.p):
            y = sp.sqrt_mod(rhs, self.p)
            return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

  def koblitz_encode_message(self, message):
    encoded_points = []
    for char in message:
      encoded_point = self.koblitz_encode(char)
      encoded_points.append(encoded_point)
      # print(f"Encoded point for character '{char}': {encoded_point}")

    return encoded_points

  def koblitz_decode(self, x):
    return (x - 1) // self.k_koblitz

  def koblitz_decode_message(self, points):
      decoded_message = []
      for point in points:
          x, y = point  # You can access both x and y here
          decoded_num = self.koblitz_decode(x)  # Decode based on x-coordinate
          decoded_message.append(decoded_num)
      return decoded_message

  def encrypt_message(self, message, public_key):
      # Encode the message into Koblitz points
      koblitz_message = self.message_to_koblitz(message)
      encoded_message = self.koblitz_encode_message(koblitz_message)
      encrypted_points = []

      # Encrypt each Koblitz point
      for point in encoded_message:
          encrypted_point = self.enryption(point, public_key)
          encrypted_points.append(encrypted_point)
      print(encrypted_points)
      # Convert encrypted_points (a list of tuples) to a JSON string
      encrypted_message_string = json.dumps(encrypted_points)

      # Optionally, encode the string to base64 for a cleaner ciphertext
      encrypted_message_base64 = base64.b64encode(encrypted_message_string.encode()).decode()

      return encrypted_message_base64  # Return as readable string

  def decrypt_message(self, encrypted_message_base64, private_key):
      # Decode the base64 string back to the JSON string
      encrypted_message_string = base64.b64decode(encrypted_message_base64).decode()

      # Convert the JSON string back to a list of encrypted points (tuples)
      encrypted_points = json.loads(encrypted_message_string)

      decrypted_message = []

      # Decrypt each point
      for point in encrypted_points:
          decrypted_point = self.decryption(point, private_key)
          decrypted_message.append(decrypted_point)

      # Decode the Koblitz points into the original numeric values
      decoded_koblitz_points = self.koblitz_decode_message(decrypted_message)

      # Convert numeric Koblitz points to characters using num_to_char
      plain_text_message = ''.join([self.num_to_char(num) for num in decoded_koblitz_points])

      return plain_text_message



# Class Initialize

In [112]:
random_num = random.randint(1, p - 1)
print(f'random number: {random_num}')

elliptic1 = EllipticCurveElGamal(a, b, p, random_num)

# k = elliptic1.generate_random_number()
# print(f'k: {k}')

basis = elliptic1.generate_random_point()
print(f'basis: {basis}')

elliptic1.PointB = basis

print(f'is basis on curve: {elliptic1.is_on_curve(basis[0], basis[1])}')

random number: 634
basis: (466, 608)
is basis on curve: True


# Key Generation

In [113]:
private_key = random.randint(1, p - 1)
public_key = elliptic1.generate_public_key(basis, private_key)

print(f'private key: {private_key}')
print(f'public key: {public_key}')

private key: 433
public key: (158, 29)


# Ecnrypt & Decrypt

In [114]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

In [115]:
# elliptic1.k_koblitz = 18 # max 18 if add "-#./*"

# plaintexts = "-#./*"
plaintext = "lorem.#-/*"
# plaintexts = CONTENT
print(f'Plaintext: {plaintext}')

ciphertext = elliptic1.encrypt_message(plaintext, public_key)
print(f"Ciphertext: {ciphertext}")

# Decrypt the ciphertext back to the original message
decrypted_message = elliptic1.decrypt_message(ciphertext, private_key)
print(f"Decrypted message: {decrypted_message}")

Plaintext: lorem.#-/*
[((161, 750), (285, 655)), ((161, 750), (562, 201)), ((161, 750), (297, 182)), ((161, 750), (187, 50)), ((161, 750), (310, 31)), ((161, 750), (373, 9)), ((161, 750), (124, 397)), ((161, 750), (102, 631)), ((161, 750), (159, 627)), ((161, 750), (331, 384))]
Ciphertext: W1tbMTYxLCA3NTBdLCBbMjg1LCA2NTVdXSwgW1sxNjEsIDc1MF0sIFs1NjIsIDIwMV1dLCBbWzE2MSwgNzUwXSwgWzI5NywgMTgyXV0sIFtbMTYxLCA3NTBdLCBbMTg3LCA1MF1dLCBbWzE2MSwgNzUwXSwgWzMxMCwgMzFdXSwgW1sxNjEsIDc1MF0sIFszNzMsIDldXSwgW1sxNjEsIDc1MF0sIFsxMjQsIDM5N11dLCBbWzE2MSwgNzUwXSwgWzEwMiwgNjMxXV0sIFtbMTYxLCA3NTBdLCBbMTU5LCA2MjddXSwgW1sxNjEsIDc1MF0sIFszMzEsIDM4NF1dXQ==
Decrypted message: 258e3h0321


# Encode & Decode Cipher Point

In [116]:
message = (385, 423)

cipher = elliptic1.enryption(message, public_key)

decrypted_message = elliptic1.decryption(cipher, private_key)

print(f'message: {message}')
print(f'cipher: {cipher}')
print(f'decrypted message: {decrypted_message}')

message: (385, 423)
cipher: ((161, 750), (325, 23))
decrypted message: (385, 423)


In [117]:
message_to_koblitz = elliptic1.message_to_koblitz("lorem")
print(message_to_koblitz)

[21, 24, 27, 14, 22]


In [118]:
message_point = elliptic1.koblitz_encode_message(message_to_koblitz)
print(message_point)

[(872, 39), (985, 188), (1109, 200), (575, 329), (903, 337)]


In [119]:
point_message = elliptic1.koblitz_decode_message(message_point)
print(point_message)

[21, 24, 27, 14, 22]


In [120]:
koblitz_to_message = elliptic1.koblitz_to_message(point_message)
print(koblitz_to_message)

lorem


In [188]:
elliptic1.k_koblitz = 3 # max 18 if add "-#./*"

chunk1 = ((636, 659), (592, 660))

point_message = elliptic1.koblitz_decode_message([chunk1[1]])
print(point_message)

message_point = elliptic1.koblitz_encode_message(point_message)
print(message_point)

[197]
[(592, 91)]


In [122]:
import math

# Base 94 encoding uses characters between ASCII 33 ('!') to 126 ('~')
BASE = 94
OFFSET = 33  # ASCII offset to map numbers to characters (starts from '!')
MAX_VALUE = BASE ** 4  # Allow 4-character encoding for larger numbers

# Helper function to encode an integer to base 94 string
def encode_number(num):
    encoded_str = ''
    while num > 0:
        encoded_str = chr((num % BASE) + OFFSET) + encoded_str
        num //= BASE
    # Pad with '!' to make it 4 characters long
    return encoded_str.rjust(4, '!')

# Helper function to decode base 94 string back to an integer
def decode_string(encoded_str):
    num = 0
    for char in encoded_str:
        num = num * BASE + (ord(char) - OFFSET)
    return num

# Function to encode elliptic curve points into characters
def encode_point_to_chars(point):
    x, y = point
    # Combine x and y into a large number by assuming x and y are within a reasonable range
    combined_num = x * MAX_VALUE + y
    # Encode the combined number into base 94 string
    return encode_number(combined_num)

# Function to decode characters back into elliptic curve points
def decode_chars_to_point(encoded_str):
    # Decode the string back into a large number
    combined_num = decode_string(encoded_str)
    # Extract x and y from the combined number
    x = combined_num // MAX_VALUE
    y = combined_num % MAX_VALUE
    return (x, y)

# Test with the given points
cipher_points = [((636, 659), (592, 660)), ((636, 659), (697, 472)),
                 ((636, 659), (635, 509)), ((636, 659), (385, 328)),
                 ((636, 659), (372, 37)), ((636, 659), (731, 529)),
                 ((636, 659), (268, 522)), ((636, 659), (629, 545)),
                 ((636, 659), (465, 570)), ((636, 659), (54, 620))]
# [(636, 659), (592, 660), (697, 472),(635, 509), (385, 328), (372, 37), (731, 529), (268, 522), (629, 545), (465, 570), (54, 620)]
# Encode each point pair into a string
encoded_ciphertext = []
for point_pair in cipher_points:
    encoded_pair = (encode_point_to_chars(point_pair[0]), encode_point_to_chars(point_pair[1]))
    encoded_ciphertext.append(encoded_pair)

# Print the encoded cipher text
print("Encoded ciphertext:")
for encoded_pair in encoded_ciphertext:
    print(encoded_pair)

# Decode back to points
decoded_points = []
for encoded_pair in encoded_ciphertext:
    decoded_pair = (decode_chars_to_point(encoded_pair[0]), decode_chars_to_point(encoded_pair[1]))
    decoded_points.append(decoded_pair)

# Print the decoded points to verify
print("\nDecoded points:")
for point_pair in decoded_points:
    print(point_pair)


Encoded ciphertext:
('\'i!!("', "'=!!(#")
('\'i!!("', '(H!!&#')
('\'i!!("', "'h!!&H")
('\'i!!("', '%*!!$O')
('\'i!!("', '${!!!F')
('\'i!!("', '(j!!&\\')
('\'i!!("', '#q!!&U')
('\'i!!("', "'b!!&l")
('\'i!!("', "%z!!''")
('\'i!!("', "W!!'Y")

Decoded points:
((636, 659), (592, 660))
((636, 659), (697, 472))
((636, 659), (635, 509))
((636, 659), (385, 328))
((636, 659), (372, 37))
((636, 659), (731, 529))
((636, 659), (268, 522))
((636, 659), (629, 545))
((636, 659), (465, 570))
((636, 659), (54, 620))


In [123]:
import math

# Base 94 encoding uses characters between ASCII 33 ('!') to 126 ('~')
BASE = 94
OFFSET = 33  # ASCII offset to map numbers to characters (starts from '!')
MAX_VALUE = BASE ** 4  # Allow 4-character encoding for larger numbers

# Helper function to encode an integer to base 94 string
def encode_number(num):
    encoded_str = ''
    while num > 0:
        encoded_str = chr((num % BASE) + OFFSET) + encoded_str
        num //= BASE
    # Pad with '!' to make it 4 characters long
    return encoded_str.rjust(4, '!')

# Helper function to decode base 94 string back to an integer
def decode_string(encoded_str):
    num = 0
    for char in encoded_str:
        num = num * BASE + (ord(char) - OFFSET)
    return num

# Function to encode elliptic curve points into characters
def encode_point_to_chars(point):
    x, y = point
    # Combine x and y into a large number by assuming x and y are within a reasonable range
    combined_num = x * MAX_VALUE + y
    # Encode the combined number into base 94 string
    return encode_number(combined_num)

# Function to decode characters back into elliptic curve points
def decode_chars_to_point(encoded_str):
    # Decode the string back into a large number
    combined_num = decode_string(encoded_str)
    # Extract x and y from the combined number
    x = combined_num // MAX_VALUE
    y = combined_num % MAX_VALUE
    return (x, y)

# Test with the given points
cipher_points = [((636, 659), (592, 660)), ((636, 659), (697, 472)),
                 ((636, 659), (635, 509)), ((636, 659), (385, 328)),
                 ((636, 659), (372, 37)), ((636, 659), (731, 529)),
                 ((636, 659), (268, 522)), ((636, 659), (629, 545)),
                 ((636, 659), (465, 570)), ((636, 659), (54, 620))]

# Encode each point pair into a string
encoded_ciphertext = []
for point_pair in cipher_points:
    encoded_pair = (encode_point_to_chars(point_pair[0]), encode_point_to_chars(point_pair[1]))
    encoded_ciphertext.append(encoded_pair)

# Print the encoded cipher text
encoded_pairs = []
print("Encoded ciphertext:")
for encoded_pair in encoded_ciphertext:
    print(encoded_pair)
    # Concatenate the tuple into a single string and add to the list
    encoded_pairs.append(''.join(encoded_pair))

# Join all encoded pairs into a single string
encoded_string = ''.join(encoded_pairs)
print(f"\nEncoded full string: {encoded_string}")

# Decode back to points
decoded_points = []
for encoded_pair in encoded_ciphertext:
    decoded_pair = (decode_chars_to_point(encoded_pair[0]), decode_chars_to_point(encoded_pair[1]))
    decoded_points.append(decoded_pair)

# Print the decoded points to verify
print("\nDecoded points:")
for point_pair in decoded_points:
    print(point_pair)


Encoded ciphertext:
('\'i!!("', "'=!!(#")
('\'i!!("', '(H!!&#')
('\'i!!("', "'h!!&H")
('\'i!!("', '%*!!$O')
('\'i!!("', '${!!!F')
('\'i!!("', '(j!!&\\')
('\'i!!("', '#q!!&U')
('\'i!!("', "'b!!&l")
('\'i!!("', "%z!!''")
('\'i!!("', "W!!'Y")

Encoded full string: 'i!!("'=!!(#'i!!("(H!!&#'i!!("'h!!&H'i!!("%*!!$O'i!!("${!!!F'i!!("(j!!&\'i!!("#q!!&U'i!!("'b!!&l'i!!("%z!!'''i!!("W!!'Y

Decoded points:
((636, 659), (592, 660))
((636, 659), (697, 472))
((636, 659), (635, 509))
((636, 659), (385, 328))
((636, 659), (372, 37))
((636, 659), (731, 529))
((636, 659), (268, 522))
((636, 659), (629, 545))
((636, 659), (465, 570))
((636, 659), (54, 620))


In [124]:
import math

# Base 94 encoding uses characters between ASCII 33 ('!') to 126 ('~')
BASE = 94
OFFSET = 33  # ASCII offset to map numbers to characters (starts from '!')
MAX_VALUE = BASE ** 4  # Allow 4-character encoding for larger numbers

# Helper function to encode an integer to base 94 string
def encode_number(num):
    encoded_str = ''
    while num > 0:
        encoded_str = chr((num % BASE) + OFFSET) + encoded_str
        num //= BASE
    # Pad with '!' to make it 4 characters long
    return encoded_str.rjust(4, '!')

# Helper function to decode base 94 string back to an integer
def decode_string(encoded_str):
    num = 0
    for char in encoded_str:
        num = num * BASE + (ord(char) - OFFSET)
    return num

# Function to encode elliptic curve points into characters
def encode_point_to_chars(point):
    x, y = point
    # Combine x and y into a large number by assuming x and y are within a reasonable range
    combined_num = x * MAX_VALUE + y
    # Encode the combined number into base 94 string
    return encode_number(combined_num)

# Function to encode elliptic curve points, showing x only once
def encode_points_with_unique_x(cipher_points):
    last_x_encoded = None
    encoded_string = ''

    for point_pair in cipher_points:
        x_encoded = encode_number(point_pair[0][0])  # Encode the x-coordinate
        y1_encoded = encode_number(point_pair[0][1])  # Encode the first y-coordinate
        y2_encoded = encode_number(point_pair[1][1])  # Encode the second y-coordinate

        # Only append x_encoded if it's different from the last x
        if x_encoded != last_x_encoded:
            encoded_string += x_encoded + y1_encoded + y2_encoded
            last_x_encoded = x_encoded
        else:
            # Only append the y-values if the x-coordinate is the same
            encoded_string += y1_encoded + y2_encoded

    return encoded_string

# Function to decode characters back into elliptic curve points
def decode_chars_to_point(encoded_str):
    # Decode the string back into a large number
    combined_num = decode_string(encoded_str)
    # Extract x and y from the combined number
    x = combined_num // MAX_VALUE
    y = combined_num % MAX_VALUE
    return (x, y)

# Test with the given points
cipher_points = [((636, 659), (592, 660)), ((636, 659), (697, 472)),
                 ((636, 659), (635, 509)), ((636, 659), (385, 328)),
                 ((636, 659), (372, 37)), ((636, 659), (731, 529)),
                 ((636, 659), (268, 522)), ((636, 659), (629, 545)),
                 ((636, 659), (465, 570)), ((636, 659), (54, 620))]

# Encode all point pairs, showing the x-value only once
encoded_string = encode_points_with_unique_x(cipher_points)

# Print the encoded string
print(f"Encoded full string with unique x: {encoded_string}")

# Decoding would require some more handling to extract the x-values back
# Let's just print the encoded string for now


Encoded full string with unique x: !!'i!!("!!(#!!("!!&#!!("!!&H!!("!!$O!!("!!!F!!("!!&\!!("!!&U!!("!!&l!!("!!''!!("!!'Y


In [190]:
# Original cipher points
cipher_points = [((636, 659), (592, 660)), ((636, 659), (697, 472)),
                 ((636, 659), (635, 509)), ((636, 659), (385, 328)),
                 ((636, 659), (372, 37)), ((636, 659), (731, 529)),
                 ((636, 659), (268, 522)), ((636, 659), (629, 545)),
                 ((636, 659), (465, 570)), ((636, 659), (54, 620))]

# Create a new list where the x-coordinate appears once, followed by all y-coordinates
result = []
x_value = cipher_points[0][0][0]  # Take the x-coordinate from the first point
result.append((x_value, cipher_points[0][0][1]))  # Append the first (x, y) pair

# Now append the rest as (y_x, y_y) without x again
for point_pair in cipher_points:
    # Append the (x, y) from the first point in the pair
    if point_pair[0][0] != x_value:
        result.append((point_pair[0][0], point_pair[0][1]))
    # Append the (y_x, y_y) from the second point in the pair
    result.append((point_pair[1][0], point_pair[1][1]))

# Print the transformed result
print(result)


[(636, 659), (592, 660), (697, 472), (635, 509), (385, 328), (372, 37), (731, 529), (268, 522), (629, 545), (465, 570), (54, 620)]


In [126]:
point_message = elliptic1.koblitz_decode_message(result)
print(point_message)

[15, 14, 16, 15, 9, 9, 17, 6, 15, 11, 1]


In [127]:
message_point = elliptic1.koblitz_encode_message(point_message)
print(message_point)

[(617, 149), (575, 329), (657, 156), (617, 149), (370, 11), (370, 11), (701, 203), (248, 236), (617, 149), (452, 278), (43, 322)]


In [193]:
import struct

# List of elliptic curve points (original data)
points = [((636, 659), (592, 660)), ((636, 659), (697, 472)),
                 ((636, 659), (635, 509)), ((636, 659), (385, 328)),
                 ((636, 659), (372, 37)), ((636, 659), (731, 529)),
                 ((636, 659), (268, 522)), ((636, 659), (629, 545)),
                 ((636, 659), (465, 570)), ((636, 659), (54, 620))]

# Function to convert point into bytes
def points_to_bytes(points):
    byte_data = b''  # Initialize an empty byte string
    for point in points:
        for coord_pair in point:
            # Pack each coordinate (x, y) as unsigned 16-bit integers (adjust as needed)
            byte_data += struct.pack('>HH', coord_pair[0], coord_pair[1])
    return byte_data

# Function to decode bytes back into points
def bytes_to_points(byte_data):
    points = []
    # Calculate the number of points in the byte data
    num_coords = len(byte_data) // 4  # Each (x, y) pair is 4 bytes (2 * 16-bit)
    for i in range(0, num_coords, 2):  # Process each pair of coordinates
        # Unpack two (x, y) pairs from 4 bytes at a time
        x1, y1 = struct.unpack('>HH', byte_data[i * 4:(i * 4) + 4])
        x2, y2 = struct.unpack('>HH', byte_data[(i * 4) + 4:(i * 4) + 8])
        points.append(((x1, y1), (x2, y2)))
    return points

# Convert points to bytes
byte_data = points_to_bytes(points)

# Print the resulting bytes
print(f"Encoded bytes: {byte_data}")

# Now decode the bytes back into points
decoded_points = bytes_to_points(byte_data)

# Print the decoded points
print(f"Decoded points: {decoded_points}")


Encoded bytes: b'\x02|\x02\x93\x02P\x02\x94\x02|\x02\x93\x02\xb9\x01\xd8\x02|\x02\x93\x02{\x01\xfd\x02|\x02\x93\x01\x81\x01H\x02|\x02\x93\x01t\x00%\x02|\x02\x93\x02\xdb\x02\x11\x02|\x02\x93\x01\x0c\x02\n\x02|\x02\x93\x02u\x02!\x02|\x02\x93\x01\xd1\x02:\x02|\x02\x93\x006\x02l'
Decoded points: [((636, 659), (592, 660)), ((636, 659), (697, 472)), ((636, 659), (635, 509)), ((636, 659), (385, 328)), ((636, 659), (372, 37)), ((636, 659), (731, 529)), ((636, 659), (268, 522)), ((636, 659), (629, 545)), ((636, 659), (465, 570)), ((636, 659), (54, 620))]
